# 탐색적데이터분석 (가설 평가 추정)

## 연구 질문과 가설
목적 : 키오스크 설치 대수 결정 및 효율적 위치 선정
변수 간 연관관계 
#(하차인원 <-> 공연장 수 <-> 예매 건수 <-> 문화예술관람)

1. 데이터 확인 (이상치 점검)
2. 가설 : 공연장 수가 많을수록 -> 예매건수도 많은가? (관람 직전 예매)
3. 가설 : 퇴근시간대 하차가 많은 역에 키오스크를 세우는 것이 효율적인가? (귀가자 타겟) - 자치구별 인구 데이터로 실제 거주지 확인
4. 가설 : 신용판매금액이 높을수록 -> 문화예술관람 비율도 높은가? (소비성향분석)
5. 가설명제 판명 (상관분석, 다중회귀분석, 주성분분석, PCA분석)
6. 시각화 및 결론 도출

#피드백
1. 대전의 키오스크 설치 전,후 매출증대에 영향이 있었는지에 대해 분석
2. 플랜P

In [22]:
import pandas as pd
import numpy as np
import matplotlib as mpl
import matplotlib.pyplot as plt
mpl.rcParams['font.family'].insert(0, 'Malgun Gothic')
pd.options.display.float_format = '{:.5f}'.format

In [40]:
df=pd.read_excel('데청캠\서울 지하철역 마스터시트.xlsx')
df.head()

,역사명,자치구,역위도,역경도,역사도로명주소,근처 공연장수,공연건수,예매건수,17~18,18~19,19~20,20~21,21~22,22~23,23~00,00~01,신용판매금액,문화예술관람
0,가락시장,송파구,37.49277,127.11790,서울특별시 송파구 송파대로 지하 257 (가락동),3,144,314120,476217,663040,488463,313033,271287,226994,119854,42662,1801,4.80000
1,가산디지털단지,금천구,37.48122,126.88260,서울특별시 금천구 벚꽃로 309 (가산동),7,52,18582,442850,517071,378598,210506,178233,158238,88510,12391,1092,7.00000
2,강남,강남구,37.49720,127.02810,서울특별시 강남구 강남대로 지하 396 (역삼동),32,717,729016,1977673,2692598,1675368,776469,616089,499771,278888,6168,2326,9.00000
3,강남구청,강남구,37.51715,127.04120,서울특별시 강남구 학동로 지하346 (삼성동),32,166,254494,252239,283839,189187,119943,101025,72930,48880,6570,2326,9.00000
4,강동,강동구,37.53588,127.13210,서울특별시 강동구 천호대로 지하1097 (천호동),3,31,30650,530373,843039,703247,421328,401196,370305,225776,26819,1518,4.10000


In [42]:
# 중복 값 제거 및 이상치 확인
df = df.drop_duplicates()
df.isna().sum()

역사명        0
자치구        0
역위도        0
역경도        0
역사도로명주소    0
근처 공연장수    0
공연건수       0
예매건수       0
17~18      0
18~19      0
19~20      0
20~21      0
21~22      0
22~23      0
23~00      0
00~01      0
신용판매금액     0
문화예술관람     0
dtype: int64

In [43]:
df.describe()
# 0 값 발견 - 데이터 이용시 불필요

,역위도,역경도,근처 공연장수,공연건수,예매건수,17~18,18~19,19~20,20~21,21~22,22~23,23~00,00~01,신용판매금액,문화예술관람
count,234.00000,234.00000,234.00000,234.00000,234.00000,234.00000,234.00000,234.00000,234.00000,234.00000,234.00000,234.00000,234.00000,234.00000,234.00000
mean,37.54664,127.00686,29.28632,262.87179,432911.59402,498203.76068,740348.52564,571299.24359,326331.03419,297800.17094,262075.34615,143826.98291,15147.00000,1566.43162,5.75897
std,0.04586,0.08300,44.58554,380.78438,724393.43439,428914.09242,646930.13610,501193.24035,292094.13577,275326.00687,245032.33525,134943.21697,19710.67176,345.65459,2.25256
min,37.46425,126.80150,0.00000,0.00000,0.00000,0.00000,0.00000,0.00000,0.00000,0.00000,0.00000,0.00000,0.00000,1092.00000,1.60000
25%,37.51030,126.94580,4.00000,25.50000,9640.75000,238006.75000,325968.25000,257218.50000,148663.50000,134078.25000,117424.50000,61693.75000,3723.00000,1320.00000,4.10000
50%,37.54625,127.01595,12.00000,92.00000,139401.00000,367104.50000,547748.50000,425903.00000,239060.50000,217372.50000,195705.00000,107862.00000,9643.00000,1467.00000,5.30000
75%,37.57208,127.06952,30.00000,317.50000,405154.25000,593588.75000,860853.25000,697011.25000,384985.00000,343550.00000,306489.75000,175291.00000,18752.00000,1631.25000,7.40000
max,37.68923,127.16530,167.00000,1464.00000,2845374.00000,2551130.00000,3906368.00000,3431465.00000,2111221.00000,2057398.00000,1830948.00000,1054985.00000,157926.00000,2336.00000,10.80000


In [44]:
print (df.index[(df['근처 공연장수'] == 0) | (df['공연건수'] == 0) | (df['예매건수'] == 0)].tolist())

[52, 89, 93, 230]


In [45]:
print (df.index[(df['17~18'] == 0) | (df['18~19'] == 0) | (df['19~20'] == 0) | (df['20~21'] == 0) | (df['21~22'] == 0) | (df['22~23'] == 0) | (df['23~00'] == 0) | (df['00~01'] == 0)].tolist())

[134]


In [46]:
na = [52,89,93,230,134]
for row in na:
    print(df.loc[row])
    print('-----------------------------')

역사명                               도봉산
자치구                               도봉구
역위도                          37.68923
역경도                         127.04610
역사도로명주소    서울특별시 도봉구 도봉로 964-40 (도봉동)
근처 공연장수                             0
공연건수                                0
예매건수                                0
17~18                          306524
18~19                          536680
19~20                          641964
20~21                          364530
21~22                          310721
22~23                          281816
23~00                          172216
00~01                           32347
신용판매금액                           1326
문화예술관람                        4.00000
Name: 52, dtype: object
-----------------------------
역사명                                방화
자치구                               강서구
역위도                          37.57729
역경도                         126.81270
역사도로명주소    서울특별시 강서구 금낭화로 지하132 (방화동)
근처 공연장수                             0
공연건수                              

In [47]:
df = df.drop(index=na, axis=0)

In [48]:
df.describe()

,역위도,역경도,근처 공연장수,공연건수,예매건수,17~18,18~19,19~20,20~21,21~22,22~23,23~00,00~01,신용판매금액,문화예술관람
count,229.00000,229.00000,229.00000,229.00000,229.00000,229.00000,229.00000,229.00000,229.00000,229.00000,229.00000,229.00000,229.00000,229.00000,229.00000
mean,37.54595,127.00732,29.92140,268.60699,442343.31878,501400.37555,742760.24454,571509.19651,326305.17031,297989.17467,262146.20087,143939.64192,15125.17467,1568.65502,5.77773
std,0.04484,0.08150,44.86092,382.92346,729432.57906,430329.04323,647393.71510,501278.85085,292159.75022,275776.30318,245423.54786,135161.65018,19852.92208,348.00024,2.27235
min,37.46425,126.80150,1.00000,1.00000,2.00000,24804.00000,28953.00000,24291.00000,15421.00000,14672.00000,11696.00000,6019.00000,322.00000,1092.00000,1.60000
25%,37.51012,126.94580,4.00000,28.00000,12916.00000,240253.00000,326191.00000,256885.00000,148510.00000,133994.00000,117388.00000,61220.00000,3705.00000,1320.00000,4.10000
50%,37.54538,127.01570,12.00000,103.00000,146566.00000,370394.00000,566842.00000,426088.00000,239406.00000,218035.00000,195847.00000,108476.00000,9605.00000,1486.00000,5.30000
75%,37.57129,127.06780,30.00000,338.00000,424847.00000,596451.00000,864904.00000,697018.00000,384987.00000,343954.00000,306701.00000,176316.00000,18668.00000,1635.00000,8.30000
max,37.67792,127.16530,167.00000,1464.00000,2845374.00000,2551130.00000,3906368.00000,3431465.00000,2111221.00000,2057398.00000,1830948.00000,1054985.00000,157926.00000,2336.00000,10.80000


## 가설 1 : 공연장 수 -> 예매건수 (관람 직전 예매)

## 가설 2 : 퇴근시간 하차인원 -> 예매건수

  ### 자치구별 인구 데이터로 실제 거주지 확인

In [132]:
import folium
import json
import re

In [159]:
m = folium.Map(location=[37.566535, 126.977969], zoom_start=10)
m

In [160]:
geo_json='https://raw.githubusercontent.com/southkorea/seoul-maps/master/kostat/2013/json/seoul_municipalities_geo_simple.json'

In [161]:
df2=pd.read_excel('데청캠\인구수.xlsx',header=1)
df2=df2[['행정구역별(읍면동)','총인구(명)']]
df2.columns=['name','values']
df2=df2[1:]
df2=df2.sort_values(by='name')
df2['name']=df2['name'].apply(lambda x: re.compile('[가-힣]+').findall(x)[0])
df2

,name,values
23,강남구,508135
25,강동구,444434
9,강북구,299535
16,강서구,564854
21,관악구,502641
5,광진구,353967
17,구로구,435442
18,금천구,249419
11,노원구,511982
10,도봉구,315979


In [162]:
folium.Choropleth(
    geo_data=geo_json,
    data=df2,
    columns=['name','values'],
    key_on='feature.properties.name',
    fill_color='BuPu',
    fill_opacity=0.8,
    line_opacity=0.3
).add_to(m)

m